# Read Teledyne RDI Pathfinder PD0s and smash them all into a NetCDF
Functions written by Joe Gradone and Eli Hunter

In [1]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os,sys
import glob
import datetime
import xarray as xr
import matplotlib,netCDF4
import struct
import math
from scipy.spatial.transform import Rotation as R

## To import functions from GitHub repository, make this path the path to where the repo exists locally
sys.path.insert(0,'/Users/joegradone/SynologyDrive/Drive/Rutgers/Research/code/GitHub/Glider_ADCP_Real_Time_Processing/')
#from glider_ADCP_real_time_processing import read_PD0, qaqc_data, process_data, write_data

from glider_ADCP_real_time_processing_EJHCOMMENT import read_PD0

rtime=datetime.datetime(2020,1,1,0,0,0)

In [2]:
## Setting time origin
rtime=datetime.datetime(2020,1,1,0,0,0)

## Set path to data
odir='./'
#idir = 'C:\\work\\glideradcp\\data\\ru33_2020_11_20_dvl\\pd0\\'
#idir ='/home/hunter/Projects/glider/glideradcp/ru33_2020_11_20_dvl/pd0/'
#idir='/home/hunter/Projects/glider/Glider_ADCP_Real_Time_Processing/'
idir = 'C:\\work\\glideradcp\\stthomas\\'

## Initializes empty variables, but for more that just the PD0 output
time=[]    
depth=[] 
pitch=[]
roll=[]
heading=[]
temp=[]      
u1=[]
u2=[]
u3=[]
u4=[] 
ei1=[]
ei2=[]
ei3=[]
ei4=[] 
c1=[]
c2=[]
c3=[]
c4=[]  
pg1=[]
pg2=[]
pg3=[]
pg4=[]       
bins=[]
O_ls=[]
G_ls=[]          
plt.ion()


# ## Main function to run through entire processing workflow
# def main(argv):
#     files=glob.glob(idir+'*.PD0')
#     files.sort(key=os.path.getmtime)
#     files=files[-3:]#Gets the last files. 
    
#     for file in files:
#         read_PD0(file)
#         process_data(U=u1,V=u2,H=35,dz=1,u_daverage=0,v_daverage=0)
#         write_data(file)  
#         plot_data()
#         plt.show()


## Read in binary PD0 file
def read_PD0(infile):
    global time,depth,pitch,roll,heading,temp,bins, bin1, sec
    global u1,u2,u3,u4
    global c1,c2,c3,c4
    global ei1,ei2,ei3,ei4
    global pg1,pg2,pg3,pg4
    print('Reading PDO file : '+infile)
    
    ## Open file and read in binary
    f=open(infile,'rb')
    dat = f.read()
    f.close()
    
    ## All this does is try to find the byte location of the first ensemble. 
    ## Usually its the first byte, but it some applications it is not.  
    ## It searches for the header ID and source ID (both '0x7f')
    ## See Chapter 8 of  PathFinder DVL Guide_Apr20.pdf 
    for [ind,tmp1] in enumerate(dat):
          if hex(dat[ind])=='0x7f' and hex(dat[ind+1]) =='0x7f':                 
                  break
    
    ## This extracts the number of bytes per ensemble. 
    nbytes=struct.unpack("h",dat[ind+2:ind+4])[0]+2
#    print('Finding Ensembles')      


    ## Find the starting byte of every ensemble. (Varaible Iens)
    ## It goes through every byte and searchs for header ID's and source ID's
    ## When one is found, the index is added to the variable Iens 
    Iens=[] ## Starting byte of each ensemble. 
    nind=0
    n=0
    for [ind,tmp1] in enumerate(dat):
          if hex(dat[ind])=='0x7f':
              n=n+1
              nbytes2=struct.unpack("h",dat[ind+2:ind+4])[0]+2  
             
              startens=ind
              tdat=dat[startens:startens+nbytes]
              if len(tdat)<nbytes:
                   print('breaking')
                   break
              tmp=tdat[nbytes-2:nbytes]
              chksum=struct.unpack("<H",tmp)[0]
              if (sum(tdat[:nbytes-2]) & 65535) ==  chksum:
                      
                   if nbytes == nbytes2:
  
                       nind=ind
                       Iens.append(ind)
#               else:
#                  print('Bad Checksum')

    nens=len(Iens)

## nens is number of ensembles, so this chunk is preallocating variables that will
## be read in. Is the 100 for the 2-dimensional variables just a "safe" number of bins
## without knowing exactly how many. There's a better way to do this...
    time=np.empty((nens),np.double)    
    depth=np.empty((nens),np.double) 
    pitch=np.empty((nens),np.double) 
    roll=np.empty((nens),np.double) 
    heading=np.empty((nens),np.double) 
    temp=np.empty((nens),np.double)
    
    u1=np.empty((nens,100),np.double) 
    u2=np.empty((nens,100),np.double) 
    u3=np.empty((nens,100),np.double)
    u4=np.empty((nens,100),np.double)  
    ei1=np.empty((nens,100),np.double) 
    ei2=np.empty((nens,100),np.double) 
    ei3=np.empty((nens,100),np.double)
    ei4=np.empty((nens,100),np.double)  
    c1=np.empty((nens,100),np.double) 
    c2=np.empty((nens,100),np.double) 
    c3=np.empty((nens,100),np.double)
    c4=np.empty((nens,100),np.double)   
    pg1=np.empty((nens,100),np.double) 
    pg2=np.empty((nens,100),np.double) 
    pg3=np.empty((nens,100),np.double)
    pg4=np.empty((nens,100),np.double)    
    xform=np.zeros((4,4),np.double)   
    xformR=np.zeros((3,3),np.double)
    xformP=np.zeros((3,3),np.double)
    xformH=np.zeros((3,3),np.double)

    ind=0
    eoffset=0
#    Iens=Iens[0:nens]

## Loop through ensembles and pull out data.
## Which bytes correspiond to what variables is detailed in the Pathfinder manual Ch 8. This is standard for PD0s?
    for ind2 in Iens:
        startens=(ind2)
        tdat=dat[startens:startens+nbytes]
        # a=buffer(tdat,2,2)
        tnbytes=struct.unpack("H",tdat[2:4])[0]+2
        # a=buffer(tdat,nbytes-2,2)
        chksum=struct.unpack("<H",tdat[nbytes-2:nbytes])[0]

        ## In the past binary data was subject to erros (missing bytes)during read/writing/data transfer operations. 
        ## It is common practive to add a Checksum to the end of the data. 
        ## i.e. the last 2 bytes of the ensemble represent the "sum" of every byte in the ensemble. 
        ## if they do not match, data was lost in the ensemble. This happens rarely.
        if (sum(tdat[:nbytes-2]) & 65535) ==  chksum:
              ndtype=struct.unpack("b",tdat[5:6])[0]
 
              offsets=list()
              for ind3 in range(ndtype):
                   Is=6+ind3*2
                   offsets.append(struct.unpack_from("h",tdat[Is:Is+2])[0])        
            
            ## FIXEDLEADER
            ## Number of beams
              Is=offsets[0]+8
              nbeam=tdat[Is]
            ## Number of cells
              Is=offsets[0]+9
              ncells=tdat[Is]  
            ## Cell size
              Is=offsets[0]+12
              cellsize=struct.unpack("H",tdat[Is:Is+2])[0]        
              cellsize=cellsize/100.0       
            ## Bin 1 distance in cm --> meters
              Is=offsets[0]+32
              bin1=struct.unpack("H",tdat[Is:Is+2])[0]    
              bin1=bin1/100.0
            ## Heading alignment/100 to get degrees    
              Is=offsets[0]+26
              hdalign=struct.unpack("H",tdat[Is:Is+2])[0]    
              hdalign=hdalign/100.0
            ## Heading bias/100 to get degrees
              Is=offsets[0]+28
              hdbias=struct.unpack("H",tdat[Is:Is+2])[0]    
              hdbias=hdbias/100.0
            
              Is=offsets[0]+4
              # sysconfig1=bin(tdat[Is])   
              sysconfig1=format(tdat[Is], '#010b')[2:]
              Is=offsets[0]+5
              # sysconfig2=bin(tdat[Is]) 
              sysconfig2=format(tdat[Is], '#010b')[2:]
            
            ## Beam angle correction
              if sysconfig2[-2:]=='10':
                  bmang=30.0
              elif sysconfig2[-2:]=='01':
                  bmang=20.0
              elif sysconfig2[-2:]=='00':
                  bmang=15.0
              a=1.0/(2.0*np.sin(bmang*np.pi/180.0))
              b=1.0/(4.0*np.cos(bmang*np.pi/180.0))
              c=1.0
              d=a/np.sqrt(2.0)
            ## Building transformation matrix for beam to instrument
              xform[0,0]=c*a  
              xform[0,1]=-c*a
              xform[0,2]=0.0
              xform[0,3]=0.0  
            
              xform[1,0]=0.0  
              xform[1,1]=0.0
              xform[1,2]=-c*a
              xform[1,3]=c*a
              
              xform[2,0]=b  
              xform[2,1]=b
              xform[2,2]=b
              xform[2,3]=b
              
              xform[3,0]=d  
              xform[3,1]=d
              xform[3,2]=-d
              xform[3,3]=-d

              Is=offsets[1]+2
              ens=struct.unpack("H",tdat[Is:Is+2])[0]        
              
              
     
            ## Read in time data
              Is=offsets[1]+4
              year=tdat[Is]  
        
              Is=offsets[1]+5
              month=tdat[Is]
              Is=offsets[1]+6
              day=tdat[Is]
              Is=offsets[1]+7
              hour=tdat[Is]
              Is=offsets[1]+8
              minute=tdat[Is]
              Is=offsets[1]+9
              sec=tdat[Is]
              Is=offsets[1]+10
              hsec=tdat[Is]

              ttime = datetime.datetime(year+2000,month,day,hour,minute,sec,hsec*10)-rtime
            ## Read in depth data
              Is=offsets[1]+16
              tdepth=struct.unpack("H",tdat[Is:Is+2])[0]   
            ## Convert depth from decimeters to meters
              tdepth=tdepth*0.1
            ## Read in temporary heading
              Is=offsets[1]+18
              theading=struct.unpack("H",tdat[Is:Is+2])[0]     
              theading=theading/100.0
            ## Read in pitch data
              Is=offsets[1]+20
              tpitch=struct.unpack("h",tdat[Is:Is+2])[0]        
              tpitch=tpitch/100.0 
            ## Read in roll data
              Is=offsets[1]+22
              troll=struct.unpack("h",tdat[Is:Is+2])[0]        
              troll=troll/100.0    
            ## Read in salinity data
              Is=offsets[1]+24
              tsalt=struct.unpack("h",tdat[Is:Is+2])[0]
            ## Read in temperature data
              Is=offsets[1]+26
              ttemp=struct.unpack("h",tdat[Is:Is+2])[0]        
              ttemp=ttemp/100.0       
            ## Read in glider pressure data
              Is=offsets[1]+48
              tpress=struct.unpack("i",tdat[Is:Is+4])[0]        
              tpress=tpress/1000.0       
            ## Read in velocity data
              Is=offsets[2]+2
              fmt = "<%dh" % (ncells*4)
              uvw=struct.unpack(fmt,tdat[Is:Is+ncells*4*2])
              uvw=np.array(uvw,dtype=float)
            ## Read in echo intensity data
              Is=offsets[3]+2
              fmt = "<%dB" % (ncells*4)
              tEI=struct.unpack(fmt,tdat[Is:Is+ncells*4])
              tEI=np.array(tEI)
            ## Read in correlation data
              Is=offsets[4]+2
              fmt = "<%dB" % (ncells*4)
              tC=struct.unpack(fmt,tdat[Is:Is+ncells*4])
              tC=np.array(tC)  
            ## Read in percent good data
              Is=offsets[5]+2
              fmt = "<%dB" % (ncells*4)
              tPG=struct.unpack(fmt,tdat[Is:Is+ncells*4])
              tPG=np.array(tPG)              
            ## Reshape velocity, ei, corr, and pg to be 2D based on cells and beams
              uvw.shape=(ncells,4)
              tEI.shape=(ncells,4)
              tC.shape=(ncells,4)
              tPG.shape=(ncells,4)
            ## Create bins variable based on number of cells and cell size and reference it to distance from sensor based on bin1
              bins=(np.arange(0,ncells,1,np.double)*cellsize)+bin1    
            ## Read in bottom track data/check if bottom track
              LO=len(offsets)
              if LO>6:
                Is=offsets[6]
                tmp1=struct.unpack("c",tdat[Is:Is+1])[0]
                Is=offsets[6]+1
                tmp2=struct.unpack("c",tdat[Is:Is+1])[0]
                if [tmp1+tmp2]==[b'\x00\x06']:
                    Is=offsets[6]+16
                    tr1=struct.unpack("H",tdat[Is:Is+2])[0]       
                    Is=offsets[6]+18
                    tr2=struct.unpack("H",tdat[Is:Is+2])[0]    
                    Is=offsets[6]+20
                    tr3=struct.unpack("H",tdat[Is:Is+2])[0]    
                    Is=offsets[6]+22
                    tr4=struct.unpack("H",tdat[Is:Is+2])[0] 
                    ## Convert bottom track range to meters
                    tr1=tr1/100.0     
                    tr2=tr2/100.0     
                    tr3=tr3/100.0     
                    tr4=tr4/100.0

                    
                
            ## Only set velocity data below detected bottom equal to 0 if the bottom track range is greater than 0
                if tr1>0:
                    uvw[bins>.85*tr1,0]=float("NAN")
                if tr2>0:
                    uvw[bins>.85*tr2,1]=float("NAN")
                if tr3>0:
                    uvw[bins>.85*tr3,2]=float("NAN")
                if tr4>0:
                    uvw[bins>.85*tr4,3]=float("NAN")
              
            ## Bin map velocity data based on pitch and roll
              uvw=mapdepthcells(uvw,tpitch,troll)
            ## This is where we actually build the arrays/vecotrs of the PD0 data. 
            ## ind is incremented at the end of the code block (ind=ind+1 below)
            ## It's how we add data to the variables we initialized earlier.  
              time[ind]=ttime.days+ttime.seconds/86400.0
                  
              depth[ind]=tdepth
              pitch[ind]=tpitch
              roll[ind]=troll
              temp[ind]=ttemp
              heading[ind]=theading  
              # P=np.arctan(np.tan(tpitch*np.pi/180.0)*np.cos(troll*np.pi/180.0))

            ## Correct heading
              shead=theading+hdalign
            ## Build beam to ENU transformation matrix
              CH=np.cos(shead*np.pi/180.0)
              SH=np.sin(shead*np.pi/180.0)
              CR=np.cos(troll*np.pi/180.0)
              SR=np.sin(troll*np.pi/180.0)
              CP=np.cos(tpitch*np.pi/180.0)
              SP=np.sin(tpitch*np.pi/180.0)
              # print(CP)
              # CP=np.cos(P)
              # print(CP)
              # SP=np.sin(P)
              
              xformH[0,0]=CH  
              xformH[0,1]=SH
              xformH[0,2]=0.0 
              xformH[1,0]=-SH 
              xformH[1,1]=CH
              xformH[1,2]=0.0 
              xformH[2,0]=0.0  
              xformH[2,1]=0.0
              xformH[2,2]=1.0 
            
              xformR[0,0]=CR  
              xformR[0,1]=0.0
              xformR[0,2]=SR 
              xformR[1,0]=0.0 
              xformR[1,1]=1.0
              xformR[1,2]=0.0 
              xformR[2,0]=-SR  
              xformR[2,1]=0.0
              xformR[2,2]=CR
              
              xformP[0,0]=1.0  
              xformP[0,1]=0.0
              xformP[0,2]=0.0 
              xformP[1,0]=0.0 
              xformP[1,1]=CP
              xformP[1,2]=-SP 
              xformP[2,0]=0.0  
              xformP[2,1]=SP
              xformP[2,2]=CP
              
            ## QAQC Data
              uvw = qaqc_data(uvw,tC,tEI,tPG)
            
            ## Convert from beam to ENU velocity
              uvw=uvw @ xform.transpose()
              terr=uvw[:,3]
              tuvw=uvw[:,0:3]
              tuvw=tuvw @ xformR.transpose()
              tuvw=tuvw @ xformP.transpose()
              tuvw=tuvw @ xformH.transpose()
            
              u1[ind,0:ncells]=tuvw[:,0] ## This is now U velocity
              u2[ind,0:ncells]=tuvw[:,1] ## This is now V velocity
              u3[ind,0:ncells]=tuvw[:,2] ## This is now W velocity
              u4[ind,0:ncells]=terr      ## This is now error velocity
              ei1[ind,0:ncells]=tEI[:,0] 
              ei2[ind,0:ncells]=tEI[:,1]
              ei3[ind,0:ncells]=tEI[:,2]
              ei4[ind,0:ncells]=tEI[:,3]
              c1[ind,0:ncells]=tC[:,0]
              c2[ind,0:ncells]=tC[:,1]
              c3[ind,0:ncells]=tC[:,2]
              c4[ind,0:ncells]=tC[:,3]
              pg1[ind,0:ncells]=tPG[:,0]
              pg2[ind,0:ncells]=tPG[:,1]
              pg3[ind,0:ncells]=tPG[:,2]
              pg4[ind,0:ncells]=tPG[:,3]            
              
              ind=ind+1
        else:
          #   print 'BAD CHECKSUM'
            # eoffset=eoffset+1
            
            continue
    
    
    u1=u1[:,0:ncells]
    u2=u2[:,0:ncells]
    u3=u3[:,0:ncells]
    u4=u4[:,0:ncells]
    c1=c1[:,0:ncells]
    c2=c2[:,0:ncells]
    c3=c3[:,0:ncells]
    c4=c4[:,0:ncells]
    ei1=ei1[:,0:ncells]
    ei2=ei2[:,0:ncells]
    ei3=ei3[:,0:ncells]
    ei4=ei4[:,0:ncells]
    pg1=pg1[:,0:ncells]
    pg2=pg2[:,0:ncells]
    pg3=pg3[:,0:ncells]
    pg4=pg4[:,0:ncells]
    


def mapdepthcells(uvw,tpitch,troll):
    global bins
 #   print('Mapping depth cells')
    brange=bins/np.cos(30*np.pi/180.0)
    tuvw=uvw*np.nan
    az=90*np.pi/180
    elev=-60*np.pi/180
    ## This is getting the actuall bin depths.(relative to glider), so we can transform them to "Level" bin depths 
    XYZ1 = sph2cart(az,elev,brange)
    
   
    az=-90*np.pi/180
    elev=-60*np.pi/180
    XYZ2 = sph2cart(az,elev,brange)
    
    az=0*np.pi/180
    elev=-60*np.pi/180
    XYZ3 = sph2cart(az,elev,brange) 
    
    az=180*np.pi/180
    elev=-60*np.pi/180
    XYZ4= sph2cart(az,elev,brange)
#    trot=np.array([[0.9330  , -0.0670 ,  -0.3536],[-0.0670  , 0.9330 ,  -0.3536],[0.3536 ,   0.3536 ,   0.8660]])


#    print([tpitch,troll])
    rang1=tpitch*np.pi/(180) 
    rang2=troll*np.pi/(180) 
    sc=1/np.sqrt(2.0)
    ax1=sc*rang1*np.array([-1, 1, 0])
    ax2=sc*rang2*np.array([1, 1, 0])
    rmx1 = R.from_rotvec(ax1)
    rmx2 = R.from_rotvec(ax2)
    
    rot1=rmx1.as_matrix()    
    rot2=rmx2.as_matrix()
    rXYZ1=np.concatenate(([XYZ1[0]],[XYZ1[1]],[XYZ1[2]]),axis=0)
    r1=rXYZ1.transpose() @ rot1 @ rot2
    rXYZ2=np.concatenate(([XYZ2[0]],[XYZ2[1]],[XYZ2[2]]),axis=0)
    r2=rXYZ2.transpose() @ rot1  @ rot2
    rXYZ3=np.concatenate(([XYZ3[0]],[XYZ3[1]],[XYZ3[2]]),axis=0)
    r3=rXYZ3.transpose() @ rot1 @ rot2
    rXYZ4=np.concatenate(([XYZ4[0]],[XYZ4[1]],[XYZ4[2]]),axis=0)
    r4=rXYZ4.transpose() @ rot1 @ rot2
    
    # gX=np.arange(-10,10)
    # gY=gX*0.0
    # g=(gX+1j*gY)*np.exp(1j*45.0*np.pi/180)
    # gX=np.real(g)
    # gY=np.imag(g)
    # gZ=0.0*gX
    
    
    # gDX=bins*0.0
    # gDY=bins*0.0
    # gDZ=-bins
    # tmp=np.concatenate(([gX],[gY],[gZ]),axis=0)
    # rg=tmp.transpose() @ rot1 @ rot2
    
    # tmp=np.concatenate(([gDX],[gDY],[gDZ]),axis=0)
    # rDg=tmp.transpose() @ rot1 @ rot2
    
    # plt.figure(1)
    # plt.clf()
    # ax = plt.axes(projection='3d')
    # ax.plot3D(XYZ1[0],XYZ1[1],XYZ1[2],'r')
    # ax.plot3D(r1[:,0],r1[:,1],r1[:,2],'b')

    # ax.plot3D(XYZ2[0],XYZ2[1],XYZ2[2],'r')
    # ax.plot3D(r2[:,0],r2[:,1],r2[:,2],'b')
    # ax.plot3D(XYZ3[0],XYZ3[1],XYZ3[2],'r')
    # ax.plot3D(r3[:,0],r3[:,1],r3[:,2],'b')
    # ax.plot3D(XYZ4[0],XYZ4[1],XYZ4[2],'r')
    # ax.plot3D(r4[:,0],r4[:,1],r4[:,2],'b')
    # ax.plot3D(gX,gY,gZ,'k')
    # ax.plot3D(rg[:,0],rg[:,1],rg[:,2],'b')
    

    # ax.plot3D(gDX,gDY,gDZ,'r')
    # ax.plot3D(rDg[:,0],rDg[:,1],rDg[:,2],'b')
    # plt.grid(True)
    
    tuvw[:,0]=np.interp(-r1[:,2],bins,uvw[:,0])
    tuvw[:,1]=np.interp(-r2[:,2],bins,uvw[:,1])
    tuvw[:,2]=np.interp(-r3[:,2],bins,uvw[:,2])
    tuvw[:,3]=np.interp(-r4[:,2],bins,uvw[:,3])
    # plt.figure(3)
    # plt.clf()
    # plt.subplot(141)
    # plt.plot(uvw[:,0],-bins,'r',marker='o')
    # plt.plot(tuvw[:,0],-bins,'r',marker='+')
    # plt.grid(True)
    # plt.subplot(142)
    # plt.plot(uvw[:,1],-bins,'b',marker='o')
    # plt.plot(tuvw[:,1],-bins,'b',marker='+')
    # plt.grid(True)
    # plt.subplot(143)
    # plt.plot(uvw[:,2],-bins,'k',marker='o')
    # plt.plot(tuvw[:,2],-bins,'k',marker='+')
    # plt.grid(True)
    # plt.subplot(144)
    # plt.plot(uvw[:,3],-bins,'g',marker='o')
    # plt.plot(tuvw[:,3],-bins,'g',marker='+')
    # plt.grid(True)
    # plt.show()
    
    # print("Paused")
    # input()
    return tuvw

def qaqc_data(uvw,tC,tEI,tPG):
    #print('PROCESSING DVL DATA')
    corr_cut = 50
    ei_cut   = 70
    pg_cut   = 80
    
    # Change filled values to NaN
    uvw[uvw[:,0] == -32768,0] = float("NAN")
    uvw[uvw[:,1] == -32768,1] = float("NAN")
    uvw[uvw[:,2] == -32768,2] = float("NAN")
    uvw[uvw[:,3] == -32768,3] = float("NAN")
    
    # Convert from mm/s to m/s
    uvw[:,0] = uvw[:,0]/1000
    uvw[:,1] = uvw[:,1]/1000
    uvw[:,2] = uvw[:,2]/1000
    uvw[:,3] = uvw[:,3]/1000    
    
    ## Filter for bad correlation
    uvw[tC[:,0] < corr_cut,0] = float("NAN")
    uvw[tC[:,1] < corr_cut,1] = float("NAN")
    uvw[tC[:,2] < corr_cut,2] = float("NAN")
    uvw[tC[:,3] < corr_cut,3] = float("NAN")
    
#     ## Filter for bad echo intensity
#     uvw[tEI[:,0] < ei_cut,0] = float("NAN")
#     uvw[tEI[:,1] < ei_cut,1] = float("NAN")
#     uvw[tEI[:,2] < ei_cut,2] = float("NAN")
#     uvw[tEI[:,3] < ei_cut,3] = float("NAN")

    ## Filter for bad percent good
    uvw[tPG[:,0] < pg_cut,0] = float("NAN")
    uvw[tPG[:,1] < pg_cut,1] = float("NAN")
    uvw[tPG[:,2] < pg_cut,2] = float("NAN")
    uvw[tPG[:,3] < pg_cut,3] = float("NAN")
    
    return(uvw)

                        
def process_data(U,V,H,dz,u_daverage,v_daverage):
    global O_ls, G_ls, bin_new    
    
    ## Feb-2021 jgradone@marine.rutgers.edu Initial
    ## Jul-2021 jgradone@marine.rutgers.edu Updates for constraints
    
    ## Purpose: Take velocity measurements from glider mounted ADCP and compute
    # shear profiles
    
    ## Outputs:
    # O_ls is the ocean velocity profile
    # G_ls is the glider velocity profile
    # bin_new are the bin centers for the point in the profiles
    # C is the constant used in the constraint equation (Not applicable for
    # real-time processing)
    
    ## Inputs:
    # dz is desired vertical resolution, should not be smaller than bin length
    # H is the max depth of the water column
    # U is measured east-west velocities from ADCP
    # V is measured north-south velocities from ADCP
    # Z is the measurement depths of U and V
    # uv_daverage is depth averaged velocity (Set to 0 for real-time)
    
    ##########################################################################        
    # Take difference between bin lengths for bin size [m]
    bin_size = np.diff(bins)[0]
    bin_num = len(bins)

    # This creates a grid of the ACTUAL depths of the ADCP bins by adding the
    # depths of the ADCP bins to the actual depth of the instrument
    [bdepth,bbins]=np.meshgrid(depth,bins)
    bin_depth = bdepth+bbins  
    Z = bin_depth

    # Set knowns from Equations 19 from Visbeck (2002) page 800
    # Maximum number of observations (nd) is given by the number of velocity
    # estimates per ping (nbin) times the number of profiles per cast (nt)
    nbin = U.shape[0]  # number of programmed ADCP bins per individual profile
    nt   = U.shape[1]  # number of individual velocity profiles
    nd   = nbin*nt      # G dimension (1) 

    # Define the edges of the bins
    bin_edges = np.arange(0,math.floor(np.max(bin_depth)),dz).tolist()

    # Check that each bin has data in it
    bin_count = np.empty(len(bin_edges)-1) # Preallocate memory
    bin_count[:] = np.NaN

    for k in np.arange(len(bin_edges))[:-1]:
        # Create index of depth values that fall inside the bin edges
        ii = np.where((bin_depth > bin_edges[k]) & (bin_depth < bin_edges[k+1]))
        bin_count[k] = len(bin_depth[ii])
        ii = []

    # Create list of bin centers    
    bin_new = [x+dz/2 for x in bin_edges[:-1]]

    # Chop off the top of profile if no data
    ind = np.argmax(bin_count > 0) # Stops at first index greater than 0
    bin_new = bin_new[ind:]        # Removes all bins above first with data
    z1 = bin_new[0]                # Depth of center of first bin with data

    # Create and populate G
    nz = len(bin_new)  # number of ocean velocities desired in output profile
    nm = nz + nt       # G dimension (2), number of unknowns
    # Let's build the corresponding coefficient matrix G 
    G = np.zeros((nd,nm))

    # Indexing of the G matrix was taken from Todd et al. 2011
    for ii in np.arange(nt):           # Number of ADCP profiles per cast
        for jj in np.arange(nbin):     # Number of measured bins per profile

            # Uctd part of matrix
            G[(nbin*(ii-1))+jj,ii] = 1

            # This will fill in the Uocean part of the matrix. It loops through
            # all Z members and places them in the proper location in the G matrix

            # Find the difference between all bin centers and the current Z value        
            dx = abs(bin_new-Z[ii,jj])

            # Find the minimum of these differences
            minx = np.nanmin(dx)

            # Finds bin_new index of the first match of Z and bin_new    
            idx = np.argmin(dx-minx)

            G[(nbin*(ii-1))+jj,nt+idx] = 1
            del dx, minx, idx


    # Reshape U and V into the format of the d column vector
    d_u = np.flip(U.transpose(),axis=0)
    d_u = d_u.flatten()
    d_v = np.flip(V.transpose(),axis=0)
    d_v = d_v.flatten()


    ##########################################################################
    ## This chunk of code containts the constraints for depth averaged currents
    ## which we likely won't be using for the real-time processing

    # Need to calculate C (Todd et al. 2017) based on our inputs 
    # This creates a row that has the same # of columns as G. The elements
    # of the row follow the trapezoid rule which is used because of the
    # extension of the first bin with data to the surface. The last entry of
    # the row corresponds to the max depth reached by the glider, any bins
    # below that should have already been removed.

    constraint = np.concatenate(([np.zeros(nt)], [z1/2], [z1/2+dz/2], [[dz]*(nz-3)], [dz/2]), axis=None)

    # To find C, we use the equation of the norm and set norm=1 because we
    # desire unity. The equation requires we take the sum of the squares of the
    # entries in constraint.

    sqr_constraint = constraint*constraint
    sum_sqr_constraint = np.sum(sqr_constraint)

    # Then we can solve for the value of C needed to maintain unity 

    C = H*(1/np.sqrt(sum_sqr_constraint))

    # This is where you would add the constraint for the depth averaged
    # velocity from Todd et al., (2011/2017)

    # OG
    du = np.concatenate(([d_u],[C*u_daverage]), axis=None)
    dv = np.concatenate(([d_v],[C*v_daverage]), axis=None)

    # Build Gstar
    # Keep this out because not using depth averaged currents
    Gstar = np.vstack((G, (C/H)*constraint))

    ##########################################################################

    # Build the d matrix
    d = list(map(complex,du, dv))

    ##### Inversion!
    ## If want to do with a sparse matrix sol'n, look at scipy
    #Gs = scipy.sparse(Gstar)
    Gs = Gstar

    ms = np.linalg.solve(np.dot(Gs.conj().transpose(),Gs),Gs.conj().transpose())

    ## This is a little clunky but I think the dot product fails because of
    ## NaN's in the d vector. So, this code will replace NaN's with 0's just
    ## for that calculation    
    sol = np.dot(ms,np.where(np.isnan(d),0,d))

    O_ls = sol[nt:]   # Ocean velocity
    G_ls = sol[0:nt]  # Glider velocity


def sph2cart(az,elev,r):
    z = r * np.sin(elev)
    rcoselev = r * np.cos(elev)
    x = rcoselev * np.cos(az)
    y = rcoselev * np.sin(az)
    return x,y,z;


In [3]:

# f=open(files[0],'rb')
# dat = f.read()
# f.close()

# for [ind,tmp1] in enumerate(dat):
#       if hex(dat[ind])=='0x7f' and hex(dat[ind+1]) =='0x7f':

#               break
# nbytes=struct.unpack("h",dat[ind+2:ind+4])[0]+2
# #    print('Finding Ensembles')      

# Iens=[]
# nind=0
# n=0
# for [ind,tmp1] in enumerate(dat):
#       if hex(dat[ind])=='0x7f' and hex(dat[ind+1]) =='0x7f':
#           n=n+1
#           nbytes2=struct.unpack("h",dat[ind+2:ind+4])[0]+2  

#           startens=ind
#           tdat=dat[startens:startens+nbytes]
#           if len(tdat)<nbytes:
#                print('breaking')
#                break
#           tmp=tdat[nbytes-2:nbytes]
#           chksum=struct.unpack("<H",tmp)[0]
#           if (sum(tdat[:nbytes-2]) & 65535) ==  chksum:

#                if nbytes == nbytes2:

#                    nind=ind
#                    Iens.append(ind)
# #             else:
# #                print('Bad Checksum')


In [4]:
# test = np.empty(ind)
# for x in np.arange(0,ind):
#     if hex(dat[x]) == '0x7f':
#         print('Good checksum')
    

## Some functions to read in data

In [6]:
## Test on one file
idir = '/Users/joegradone/Desktop/RU36/ADCP/'
#idir = idir = '/Users/joegradone/Desktop/20220322_MastersOpOc/Masters_0322/Station_0/'
#idir = '/Users/joegradone/Desktop/'


files=glob.glob(idir+'*.pd0')
files.sort(key=os.path.getmtime)
read_PD0(files[0])
#read_PD0(files[60]) ## Good example of 2 meter bins to 1000 meters

#read_PD0(files[12])  ## last 1 meter bin to ~1000 meters
#read_PD0(files[15]) ## 15 is last file with 1 meter bins (500 meter dive)


Reading PDO file : /Users/joegradone/Desktop/RU36/ADCP/vb221539.pd0
breaking


In [7]:
files[60]

'/Users/joegradone/Desktop/RU36/ADCP/vb281441.pd0'

In [ ]:
len(u1)

In [ ]:
bins

In [ ]:
bin1-0.88

In [ ]:
time2 = pd.to_datetime(time, origin='2020-01-01', unit='D')
np.max(np.diff(time2))

In [ ]:
[x,y]=np.meshgrid(time,bins)
[bdepth,bbins]=np.meshgrid(depth,bins)

by=bdepth+bbins
#minind= 40
#maxind= 240

#minind=1100
#maxind=1160

#minind=2100
#maxind=2160

minind = 3800
maxind = 3850

fig = plt.figure(figsize=(20,10))
plt.pcolormesh(time[minind:maxind],by[:,minind:maxind],pg1[minind:maxind,:].transpose())
plt.plot(time[minind:maxind],depth[minind:maxind],'black',linewidth=3)
plt.ylabel('Depth [m]')
plt.gca().invert_yaxis()
plt.colorbar(label='Percent Good')
#plt.ylim(655,633)
#plt.title('RU36 Pathfinder USVI 2022 Pre-QAQC')

In [ ]:
[x,y]=np.meshgrid(time,bins)
[bdepth,bbins]=np.meshgrid(depth,bins)

by=bdepth+bbins
#minind= 40
#maxind= 240

#minind=1100
#maxind=1160

#minind=2100
#maxind=2160

minind = 3800
maxind = 3850


fig = plt.figure(figsize=(20,10))
plt.pcolormesh(time[minind:maxind],by[:,minind:maxind],c1[minind:maxind,:].transpose(),vmin=45,vmax=90)
plt.plot(time[minind:maxind],depth[minind:maxind],'black',linewidth=3)
plt.ylabel('Depth [m]')
plt.gca().invert_yaxis()
plt.colorbar(label='Correlation')
#plt.ylim(655,633)
#plt.title('RU36 Pathfinder USVI 2022 Pre-QAQC')

In [ ]:
[x,y]=np.meshgrid(time,bins)
[bdepth,bbins]=np.meshgrid(depth,bins)

by=bdepth+bbins

#minind= 40
#maxind= 240

#minind=1100
#maxind=1160

#minind=2100
#maxind=2160

minind = 3800
maxind = 3850


fig = plt.figure(figsize=(20,10))
plt.pcolormesh(time[minind:maxind],by[:,minind:maxind],ei1[minind:maxind,:].transpose())
plt.plot(time[minind:maxind],depth[minind:maxind],'black',linewidth=3)
plt.ylabel('Depth [m]')
plt.gca().invert_yaxis()
plt.colorbar(label='Echo Intensity')
#plt.ylim(655,633)
#plt.title('RU36 Pathfinder USVI 2022 Pre-QAQC')

In [ ]:
[x,y]=np.meshgrid(time,bins)
[bdepth,bbins]=np.meshgrid(depth,bins)

by=bdepth+bbins

#minind= 40
#maxind= 240

minind=1100
maxind=1160

# minind=2100
# maxind=2160

#minind = 3800
#maxind = 3850

fig = plt.figure(figsize=(20,10))
plt.pcolormesh(time[minind:maxind],by[:,minind:maxind],u2[minind:maxind,:].transpose(),vmin=-0.5,vmax=0.5)
plt.plot(time[minind:maxind],depth[minind:maxind],'black',linewidth=3)
plt.ylabel('Depth [m]')
plt.gca().invert_yaxis()
plt.colorbar(label='North-South Velocity [m/s]')
#plt.ylim(354,348)
#plt.title('RU36 Pathfinder USVI 2022 Pre-QAQC')

In [ ]:
                       
# def process_data(U,V,H,dz,u_daverage,v_daverage):
#     global O_ls, G_ls, bin_new    
    
#     ## Feb-2021 jgradone@marine.rutgers.edu Initial
#     ## Jul-2021 jgradone@marine.rutgers.edu Updates for constraints
    
#     ## Purpose: Take velocity measurements from glider mounted ADCP and compute
#     # shear profiles
    
#     ## Outputs:
#     # O_ls is the ocean velocity profile
#     # G_ls is the glider velocity profile
#     # bin_new are the bin centers for the point in the profiles
#     # C is the constant used in the constraint equation (Not applicable for
#     # real-time processing)
    
#     ## Inputs:
#     # dz is desired vertical resolution, should not be smaller than bin length
#     # H is the max depth of the water column
#     # U is measured east-west velocities from ADCP
#     # V is measured north-south velocities from ADCP
#     # Z is the measurement depths of U and V
#     # uv_daverage is depth averaged velocity (Set to 0 for real-time)
    
#     ##########################################################################        
#     # Take difference between bin lengths for bin size [m]
#     bin_size = np.diff(bins)[0]
#     bin_num = len(bins)

#     # This creates a grid of the ACTUAL depths of the ADCP bins by adding the
#     # depths of the ADCP bins to the actual depth of the instrument
#     [bdepth,bbins]=np.meshgrid(depth,bins)
#     bin_depth = bdepth+bbins  
#     Z = bin_depth

#     # Set knowns from Equations 19 from Visbeck (2002) page 800
#     # Maximum number of observations (nd) is given by the number of velocity
#     # estimates per ping (nbin) times the number of profiles per cast (nt)
#     nbin = U.shape[0]  # number of programmed ADCP bins per individual profile
#     nt   = U.shape[1]  # number of individual velocity profiles
#     nd   = nbin*nt      # G dimension (1) 

#     # Define the edges of the bins
#     bin_edges = np.arange(0,math.floor(np.max(bin_depth)),dz).tolist()

#     # Check that each bin has data in it
#     bin_count = np.empty(len(bin_edges)-1) # Preallocate memory
#     bin_count[:] = np.NaN

#     for k in np.arange(len(bin_edges))[:-1]:
#         # Create index of depth values that fall inside the bin edges
#         ii = np.where((bin_depth > bin_edges[k]) & (bin_depth < bin_edges[k+1]))
#         bin_count[k] = len(bin_depth[ii])
#         ii = []

#     # Create list of bin centers    
#     bin_new = [x+dz/2 for x in bin_edges[:-1]]

#     # Chop off the top of profile if no data
#     ind = np.argmax(bin_count > 0) # Stops at first index greater than 0
#     bin_new = bin_new[ind:]        # Removes all bins above first with data
#     z1 = bin_new[0]                # Depth of center of first bin with data

#     # Create and populate G
#     nz = len(bin_new)  # number of ocean velocities desired in output profile
#     nm = nz + nt       # G dimension (2), number of unknowns
#     # Let's build the corresponding coefficient matrix G 
#     G = np.zeros((nd,nm))

#     # Indexing of the G matrix was taken from Todd et al. 2011
#     for ii in np.arange(nt):           # Number of ADCP profiles per cast
#         for jj in np.arange(nbin):     # Number of measured bins per profile

#             if np.isfinite(U[jj,ii]):
#                 # Uctd part of matrix
#                 G[(nbin*(ii-1))+jj,ii] = 1

#                 # This will fill in the Uocean part of the matrix. It loops through
#                 # all Z members and places them in the proper location in the G matrix

#                 # Find the difference between all bin centers and the current Z value        
#                 dx = abs(bin_new-Z[ii,jj])

#                 # Find the minimum of these differences
#                 minx = np.nanmin(dx)

#                 # Finds bin_new index of the first match of Z and bin_new    
#                 idx = np.argmin(dx-minx)

#                 G[(nbin*(ii-1))+jj,nt+idx] = 1
#                 del dx, minx, idx


#     # Reshape U and V into the format of the d column vector
#     d_u = np.flip(U.transpose(),axis=0)
#     d_u = d_u.flatten()
#     d_v = np.flip(V.transpose(),axis=0)
#     d_v = d_v.flatten()


#     ##########################################################################
#     ## This chunk of code containts the constraints for depth averaged currents
#     ## which we likely won't be using for the real-time processing

#     # Need to calculate C (Todd et al. 2017) based on our inputs 
#     # This creates a row that has the same # of columns as G. The elements
#     # of the row follow the trapezoid rule which is used because of the
#     # extension of the first bin with data to the surface. The last entry of
#     # the row corresponds to the max depth reached by the glider, any bins
#     # below that should have already been removed.

#     constraint = np.concatenate(([np.zeros(nt)], [z1/2], [z1/2+dz/2], [[dz]*(nz-3)], [dz/2]), axis=None)

#     # To find C, we use the equation of the norm and set norm=1 because we
#     # desire unity. The equation requires we take the sum of the squares of the
#     # entries in constraint.

#     sqr_constraint = constraint*constraint
#     sum_sqr_constraint = np.sum(sqr_constraint)

#     # Then we can solve for the value of C needed to maintain unity 

#     C = H*(1/np.sqrt(sum_sqr_constraint))

#     # This is where you would add the constraint for the depth averaged
#     # velocity from Todd et al., (2011/2017)

#     # OG
#     du = np.concatenate(([d_u],[C*u_daverage]), axis=None)
#     dv = np.concatenate(([d_v],[C*v_daverage]), axis=None)

#     # Build Gstar
#     # Keep this out because not using depth averaged currents
#     Gstar = np.vstack((G, (C/H)*constraint))

#     ##########################################################################

#     # Build the d matrix
#     d = list(map(complex,du, dv))

#     ##### Inversion!
#     ## If want to do with a sparse matrix sol'n, look at scipy
#     #Gs = scipy.sparse(Gstar)
#     Gs = Gstar

#     ms = np.linalg.solve(np.dot(Gs.conj().transpose(),Gs),Gs.conj().transpose())

#     ## This is a little clunky but I think the dot product fails because of
#     ## NaN's in the d vector. So, this code will replace NaN's with 0's just
#     ## for that calculation    
#     sol = np.dot(ms,np.where(np.isnan(d),0,d))

#     O_ls = sol[nt:]   # Ocean velocity
#     G_ls = sol[0:nt]  # Glider velocity


# Inversion Function Edits
1) H = math.ceil(np.max(bin_depth)) and not callable in the main function <br>
2) bin_edges = np.arange(0,H+dz,dz).tolist() <br>
3) Need to make sure velocity input to function is in the correct shape, probably best to assume the function is getting the correct shape and manipulate the variable outside <br>

In [ ]:
# U = np.flip(u1.transpose())
# V = np.flip(u2.transpose())
# u_daverage = 0
# v_daverage = 0
# dz=5

# # Take difference between bin lengths for bin size [m]
# bin_size = np.diff(bins)[0]
# bin_num = len(bins)

# # This creates a grid of the ACTUAL depths of the ADCP bins by adding the
# # depths of the ADCP bins to the actual depth of the instrument
# [bdepth,bbins]=np.meshgrid(depth,bins)
# bin_depth = bdepth+bbins  
# H = math.ceil(np.max(bin_depth)) # Max sampling depth

# # Define the edges of the bins
# bin_edges = np.arange(0,H+dz,dz).tolist()

# # Check that each bin has data in it
# bin_count = np.empty(len(bin_edges)-1) # Preallocate memory
# bin_count[:] = np.NaN

# for k in np.arange(len(bin_edges))[:-1]:
#     # Create index of depth values that fall inside the bin edges
#     ii = np.where((bin_depth > bin_edges[k]) & (bin_depth < bin_edges[k+1]))
#     bin_count[k] = len(bin_depth[ii])
#     ii = []

# ## Create list of bin centers    
# bin_new = [x+dz/2 for x in bin_edges[:-1]]

# ## If there's no data at the top of the profile, chop off the top!
# ind = np.argmax(bin_count > 0) # Stops at first index greater than 0
# bin_new = bin_new[ind:]        # Removes all bins above first with data
# z1 = bin_new[0]                # Depth of center of first bin with data

# ## Create and populate G per Visbeck (2002) page 800
# nbin = U.shape[0]        # number of programmed ADCP bins
# nt   = U.shape[1]        # number of individual velocity pings
# nd   = nbin*nt           # G dimension (1) 
# nz   = len(bin_new)      # number of ocean velocities desired in output profile
# nm   = nz + nt           # G dimension (2), number of unknowns
# G    = np.zeros((nd,nm)) # Build the corresponding coefficient matrix G 

# # Indexing of the G matrix was taken from Todd et al. 2011
# for ii in np.arange(nt):        # Loop through individual pings
#     for jj in np.arange(nbin):  # Loop through individual bins

#         if np.isfinite(U[jj,ii]):
#             # Uctd part of matrix
#             G[(nbin*ii)+jj,ii] = -1
            
#             # This will fill in the Uocean part of the matrix. It loops through
#             # all Z members and places them in the proper location in the G matrix

#             # Find the difference between all bin centers and the current Z value        
#             dx = abs(bin_new-bin_depth[jj,ii])

#             # Find the minimum of these differences
#             minx = np.nanmin(dx)

#             # Finds bin_new index of the first match of Z and bin_new    
#             k = np.argmin(dx-minx)

#             G[(nbin*(ii))+jj,nt+k] = 1
#             del dx, minx, k


# # Reshape U and V into the format of the d column vector
# d_u = U.flatten()
# d_v = V.flatten()

# # Build the d matrix
# d = list(map(complex,d_u, d_v))


# # Import required package
# import numpy as np
# from scipy.sparse import csr_matrix
  
# # Creating a 3 * 4 sparse matrix
# Gstar = csr_matrix(G).toarray()

# sol = np.linalg.lstsq(Gstar,d)


In [ ]:
process_data(U=u1,V=u2,H=1000,dz=10,u_daverage=0,v_daverage=0)

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.plot(np.real(O_ls),bin_new,label='E-W Velocity',linewidth=3)
plt.plot(np.imag(O_ls),bin_new,label='N-S Velocity',linewidth=3)
plt.gca().invert_yaxis()
plt.legend()